In [52]:
import os
import hashlib
import marshal
from pyeoskit import eosapi, wallet, db
from pyeoskit import config

db.reset()
config.main_token = 'UUOS'

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')

wallet.import_key('test', '5Ji6xuMzbv8UsvJxqFzzknBhWJQZKAjqAKcxnRtMehViNfS1CSS')
wallet.import_key('test', '5Hu5ZbnRBBDAd7WA1m2DuJsN26eRtLcCEXQdvQZGCXPwCt8B3LB')
wallet.import_key('test', '5KXKjmsJNHwjGEEKfEouuSdH8vy28AHcSJbexYLngwQcUVScHSG')

def publish_contract(account_name, code, abi):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        eosapi.set_contract(account_name, code, abi, 1)
    return True

#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])

def run_test_code(code, account_name='test11111111'):
    publish_contract(account_name, code, abi)
    try:
        r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'active'})
        print(r['processed']['action_traces'][0]['console'])
    except Exception as e:
        print(e)
print('done!')

done!


```python
def token_create( issuer: int, maximum_supply: int, symbol: int) -> None
def token_issue( to: int, quantity: int, symbol: int, memo: Union[str, bytes]) -> None
def token_transfer( from_: int, to: int, quantity: int, symbol: int, memo: Union[str, bytes]) -> None
```


In [ ]:
eosapi.get_account('helloworld11')

In [53]:
account_name = 'helloworld11'
a = {
    "account": account_name,
    "permission": "active",
    "parent": "owner",
    "auth": {
        "threshold": 1,
        "keys": [
            {
                "key": "EOS6DW7yDB68zSD7GbUycAkHk6XAxwxfuHMHPJBuz2YexsdnjSZvL",
                "weight": 1
            }
        ],
        "accounts": [],
        "waits": []
    }
}
r = eosapi.push_action('eosio', 'updateauth', a, {account_name:'owner'})
print('done!')

done!


In [54]:
name = 'helloworld11'
abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [{
      "new_type_name": "account_name",
      "type": "name"
   }],
  "structs": [{
      "name": "transfer",
      "base": "",
      "fields": [
        {"name":"from", "type":"account_name"},
        {"name":"to", "type":"account_name"},
        {"name":"quantity", "type":"asset"},
        {"name":"memo", "type":"string"}
      ]
    },{
     "name": "create",
     "base": "",
     "fields": [
        {"name":"issuer", "type":"account_name"},
        {"name":"maximum_supply", "type":"asset"}
     ]
  },{
     "name": "issue",
     "base": "",
     "fields": [
        {"name":"to", "type":"account_name"},
        {"name":"quantity", "type":"asset"},
        {"name":"memo", "type":"string"}
     ]
  },{
      "name": "account",
      "base": "",
      "fields": [
        {"name":"balance", "type":"asset"}
      ]
    },{
      "name": "currency_stats",
      "base": "",
      "fields": [
        {"name":"supply", "type":"asset"},
        {"name":"max_supply", "type":"asset"},
        {"name":"issuer", "type":"account_name"}
      ]
    }
  ],
  "actions": [{
      "name": "transfer",
      "type": "transfer",
      "ricardian_contract": ""
    },{
      "name": "issue",
      "type": "issue",
      "ricardian_contract": ""
    }, {
      "name": "create",
      "type": "create",
      "ricardian_contract": ""
    }

  ],
  "tables": [{
      "name": "accounts",
      "type": "account",
      "index_type": "i64",
      "key_names" : ["currency"],
      "key_types" : ["uint64"]
    },{
      "name": "stat",
      "type": "currency_stats",
      "index_type": "i64",
      "key_names" : ["currency"],
      "key_types" : ["uint64"]
    }
  ],
  "ricardian_clauses": [],
  "abi_extensions": []
}
'''
code = '''
# -*- coding: utf-8 -*- 
import struct

def create():
    data = read_action_data()
    issuer, maximum_supply, symbol = struct.unpack('QQQ', data)
    token_create(issuer, maximum_supply, symbol)

def issue():
    data = read_action_data()
    to, quantity, symbol = struct.unpack('QQQ', data[:24])
    memo = data[25:]
#    token_issue(to, quantity, symbol, memo)
    token_issue(to, quantity, symbol, 'hello,world')

def transfer():
    data = read_action_data()
    _from, _to, quantity, symbol = struct.unpack('QQQQ', data[:32])
    memo = data[33:]
    token_transfer(_from, _to, quantity, symbol, memo)

def apply(receiver, code, action):
    if receiver != code:
        return
    if action == N('create'):
        create()
    elif action == N('issue'):
        issue()
    elif action == N('transfer'):
        transfer()
'''
publish_contract(name, code, abi)

True

In [55]:
msg = {"issuer":"helloworld11","maximum_supply":"11000000000000.0000 NNN"}
r = eosapi.push_action('helloworld11', 'create', msg, {'helloworld11':'active'})

In [56]:
'''
{"name":"to", "type":"account_name"},
{"name":"quantity", "type":"asset"},
{"name":"memo", "type":"string"}
'''

args = {"to":"helloworld11","quantity":"1000000.0000 NNN", "memo":"issue 1000000.0000 NNN to helloword11"}
#msg = {"to":"helloworld11","quantity":"1000000.0000 SSS", "memo":"hello"}
r = eosapi.push_action('helloworld11', 'issue', args, {'helloworld11':'active'})
print(r['processed']['action_traces'][0]['console'])

In [57]:
eosapi.get_balance('helloworld11', token_account='helloworld11', token_name='NNN')

1000000.0

In [58]:
args = {'from':'helloworld11', 'to':'eosio', 'quantity':'10.0000 NNN', 'memo':'hello,world'}
r = eosapi.push_action('helloworld11', 'transfer', args, {'helloworld11':'active'})

In [59]:
eosapi.get_balance('eosio', token_account='helloworld11', token_name='NNN')

10.0

```python
eosapi.set_contract(account_name, wasm_code, abi, 0)
```